In [12]:
import tushare as ts
import seaborn as sns
import keras
import numpy as np
import pandas as pd
import random
# import jtplot submodule from jupyterthemes
# from jupyterthemes import jtplot

# # currently installed theme will be used to
# # set plot style if no arguments provided
# jtplot.style()
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [2]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

code_df = pro.stock_basic(exchange='', list_status='L', fields='ts_code')

C:\Users\GreenWood\Anaconda3\envs\tf_gpu_keras\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [3]:
code_df.head()

,ts_code
0,000001.SZ
1,000002.SZ
2,000004.SZ
3,000005.SZ
4,000006.SZ


In [4]:
code_df.values.shape

(3664, 1)

In [5]:
pledge = pd.read_csv('data/pledge_clean.csv', index_col=0)
pledge['close_position_date'] = pd.to_datetime(pledge['close_position_date'])
pledge['pledge_date'] = pd.to_datetime(pledge['pledge_date'])
pledge = pledge.sort_values(by=['close_position_date'])
pledge = pledge.reset_index(drop=True)

In [6]:
pledge.head()

,code,pledge_date,alarm_line,close_position_line,price_on_pledge,total_pledge_nb,total_share_ratio,total_stock_ratio,close_position_date,is_st
0,002280.SZ,2016-03-30,16.13,14.12,25.21,4471.250000,39.53000,2.050000,2016-12-14,0
1,002280.SZ,2016-03-30,16.13,14.12,25.21,4471.250000,39.53000,2.050000,2016-12-14,0
2,002280.SZ,2016-03-30,16.13,14.12,25.21,4471.250000,39.53000,2.050000,2016-12-14,0
3,000796.SZ,2016-04-28,17.42,15.24,27.22,37910.007785,82.15942,18.559834,2016-12-20,0
4,000796.SZ,2016-04-28,17.42,15.24,27.22,37910.007785,82.15942,18.559834,2016-12-20,0


In [7]:
code_price_dict = dict()
for i in range(len(code_df)):
    code = code_df.iloc[i, 0]
    price_df = pd.read_csv('data/price/{}.csv'.format(code), index_col=0)
    price_df.index = pd.to_datetime(price_df.index)
    code_price_dict[code] = price_df

In [24]:
def get_stock_price_by_code(code, lookback, delay, pledge_date, close_position_date, code_price_dict):
    """
    通过股票代码，过去天数，生成三维的股票价格矩阵
    delay * lookback * 9(features)
    """
    non_close_rate = 4
    price_df = code_price_dict[code]
    price_df = price_df[pledge_date: close_position_date]
    price_values = price_df.values
    if len(price_df) < lookback + delay:
        print('Not good')
    close_list = [price_values[-lookback - i: -i] for i in range(1, delay + 1)]
    usable_non_close_len = len(price_df) - delay - lookback
    usable_non_close_len = min(usable_non_close_len, non_close_rate * delay)
    non_close_list = [price_values[-lookback - i: -i] for i in range(delay+1, delay+1+usable_non_close_len)]
    return close_list, non_close_list

In [25]:
close_list, non_close_list = get_stock_price_by_code(code=pledge.loc[0]['code'], 
                               lookback=60, 
                               delay=30, 
                               pledge_date=pledge.loc[0]['pledge_date'],
                               close_position_date=pledge.loc[0]['close_position_date'], 
                               code_price_dict=code_price_dict)

Not good


In [26]:
close_list.shape

AttributeError: 'list' object has no attribute 'shape'

In [15]:
non_close_list.shape

(63, 60, 9)

In [11]:
pledge.drop(['code', 'pledge_date', 'close_position_date'], axis=1).head()

,alarm_line,close_position_line,price_on_pledge,total_pledge_nb,total_share_ratio,total_stock_ratio,is_st
0,16.13,14.12,25.21,4471.250000,39.53000,2.050000,0
1,16.13,14.12,25.21,4471.250000,39.53000,2.050000,0
2,16.13,14.12,25.21,4471.250000,39.53000,2.050000,0
3,17.42,15.24,27.22,37910.007785,82.15942,18.559834,0
4,17.42,15.24,27.22,37910.007785,82.15942,18.559834,0


In [21]:
def generator(pledge, lookback, delay, min_index, max_index, code_price_dict):
    # batch_size == delay(30)
    i = min_index
    while 1:
        # loop back to min_index
        if i >= max_index:
            i = min_index
        single_pledge = pledge.loc[i]
        close_list, non_close_list = get_stock_price_by_code(code=single_pledge['code'], 
                               lookback=lookback, 
                               delay=delay, 
                               pledge_date=single_pledge['pledge_date'],
                               close_position_date=single_pledge['close_position_date'], 
                               code_price_dict=code_price_dict)
        single_pledge = single_pledge.drop(['code', 'pledge_date', 'close_position_date'])
        close_zip = [(prices, single_pledge.values, 1) for prices in close_list]
        non_close_zip = [(prices, single_pledge.values, 0) for prices in non_close_list]
        all_zip = close_zip + non_close_zip
        random.shuffle(all_zip)
        price_time = np.stack([t[0] for t in all_zip])
        pledge_info = np.stack([t[1] for t in all_zip])
        targets = np.stack([t[2] for t in all_zip])
        i += 1
        return [price_time, pledge_info], targets

In [22]:
train_gen = generator(pledge, 60, 30, 0, 500, code_price_dict)

(array([[ 2.15933000e+01,  2.15933000e+01,  2.09260000e+01,
         2.15933000e+01,  1.96312000e+01,  1.96210000e+00,
         9.99480000e+00,  5.80404310e+05,  1.25568006e+06],
       [ 2.14639000e+01,  2.14639000e+01,  1.99101000e+01,
         2.00296000e+01,  2.15933000e+01, -1.56370000e+00,
        -7.24160000e+00,  7.15524810e+05,  1.47707218e+06],
       [ 1.96013000e+01,  1.97707000e+01,  1.89539000e+01,
         1.91830000e+01,  2.00296000e+01, -8.46600000e-01,
        -4.22670000e+00,  3.75304630e+05,  7.26292573e+05],
       [ 1.91731000e+01,  1.95117000e+01,  1.90336000e+01,
         1.90834000e+01,  1.91830000e+01, -9.96000000e-02,
        -5.19200000e-01,  2.37469450e+05,  4.58645764e+05],
       [ 1.89739000e+01,  1.91033000e+01,  1.82368000e+01,
         1.83762000e+01,  1.90834000e+01, -7.07200000e-01,
        -3.70580000e+00,  3.00886960e+05,  5.59424042e+05],
       [ 1.84460000e+01,  1.85655000e+01,  1.82667000e+01,
         1.85157000e+01,  1.83762000e+01,  1.39500

ValueError: all input arrays must have the same shape